In [307]:
import numpy as np
import scipy
import quaternion as quat
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd

%matplotlib inline

COLOR = 'k'#'#FFFAF1'
plt.rcParams['font.size'] = 20
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

plt.rcParams['xtick.major.width'] = 3
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['xtick.major.size']  = 14 #12
plt.rcParams['ytick.major.size']  = 14#12

plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.minor.size']  = 8
plt.rcParams['ytick.minor.size']  = 8

plt.rcParams

plt.rcParams['axes.linewidth'] = 3

plt.rcParams['text.color'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
#plt.rcParams['axes.spines.top'] = False
#plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['axes.edgecolor'] = COLOR
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['legend.facecolor'] = 'none'
from matplotlib.gridspec import GridSpec

In [308]:
plotA=np.genfromtxt('../Lightcurves/Photometry/plotA.csv',comments='#', delimiter=',' )
plotB=np.genfromtxt('../Lightcurves/Photometry/plotB.csv',comments='#', delimiter=',' )
plotC=np.genfromtxt('../Lightcurves/Photometry/plotC.csv',comments='#', delimiter=',' )
plotD=np.genfromtxt('../Lightcurves/Photometry/plotD.csv',comments='#', delimiter=',' )
plotE=np.genfromtxt('../Lightcurves/Photometry/plotE.csv',comments='#', delimiter=',' )
plotF=np.genfromtxt('../Lightcurves/Photometry/plotF.csv',comments='#', delimiter=',' )
plotG=np.genfromtxt('../Lightcurves/Photometry/plotG.csv',comments='#', delimiter=',' )
plotH=np.genfromtxt('../Lightcurves/Photometry/plotH.csv',comments='#', delimiter=',' )
plotI=np.genfromtxt('../Lightcurves/Photometry/plotI.csv',comments='#', delimiter=',' )

belton=pd.read_csv("../Lightcurves/Photometry/1I_2017U1_lightcurve.csv")

In [2]:
def gr_indicator(chain, gr_threshold=1.01):
    chain = np.array(chain)
    (nsamp,nchain,ndim)=chain.shape   
    sw = np.mean(np.var(chain,axis=0),axis=0)   
    chainmean = np.mean(chain,axis=0)
    sv = (nsamp-1)/nsamp * sw + np.var(chainmean,axis=0)   
    RGR = sv/sw
    return(np.max(RGR)<gr_threshold)

def sqrtpdf(x,args):
    a=args
    x=np.array([x])
    y=1/np.sqrt(x)
    y[np.where(x<(1/a))]=0
    y[np.where(x>a)]=0
    return(y) 

def randDraw(pdf,args,a=100,b=None,dn=1):
    if b is None: b=a;a=-b
    assert(a<b)
    from scipy.integrate import quad
    from scipy.interpolate import interp1d
    import warnings
    import scipy

    warnings.simplefilter('ignore',scipy.integrate.IntegrationWarning) #ignores the rankwarnings from polyfit
    xsamp=np.arange(a,b,dn)
    A=1/quad(pdf,-np.infty,np.infty,args)[0]
    CDF=[]
    for x in xsamp:
        CDF.append(A*quad(pdf,-np.infty,x,args)[0])

    inverse=interp1d(CDF,xsamp)
    return(GenRand(inverse))

class GenRand:
    import numpy as np
    def __init__(self, inv):
        self.inv=inv      
    def __call__(self, size):
        x=np.random.uniform(size=int(size))
        return(self.inv(x))

In [303]:
def evolving_axis_lightcurve(theta,phi,psi,a,b,c,alpha,beta,sun=[1,0,0]): 
    import warnings
    warnings.filterwarnings("ignore")
    
    sun=quat.from_vector_part(sun)
    
    rot=np.array([np.cos(phi),np.sin(phi)*np.cos(psi),np.sin(phi)*np.sin(psi)]).T
    rot=np.einsum('ijk,ij->ijk',rot[:,None,:],beta)
    rot=quat.from_rotation_vector(rot)
    
    THETA,ALPHA=np.meshgrid(theta,alpha,indexing='ij')
    
    obs=np.array([np.cos(ALPHA),np.sin(ALPHA)*np.cos(THETA),np.sin(ALPHA)*np.sin(THETA)])
    obs=np.einsum('ijk->jki',obs)
    
    obs=quat.from_vector_part(obs)
    obs=quat.as_vector_part(rot*obs*rot.conj())
    
    sun=quat.as_vector_part(rot*sun*rot.conj())
    
    C=np.array([1/a**2,1/b**2,1/c**2])
    Ts=np.sqrt(np.einsum('ijk,ijk,k->ij',sun,sun,C))
    To=np.sqrt(np.einsum('ijk,ijk,k->ij',obs,obs,C))
    
    cosa=np.einsum('ijk,ijk,k->ij',sun,obs,C)/(Ts*To)
    cosa[np.where(cosa>1)]=1
    cosa[np.where(cosa<-1)]=-1
    
    aprime=np.arccos(cosa)
    
    aprime[np.isclose(aprime,0)]=0
    
    T=np.sqrt(np.abs(Ts**2+To**2+2*Ts*To*cosa))
    
    cosl=np.nan_to_num((Ts+To*cosa)/T,posinf=0,neginf=0)
    sinl=np.nan_to_num((To*np.sin(aprime))/T,posinf=0,neginf=0)

    lam=np.where(sinl>=0,np.arccos(cosl),-np.arccos(cosl)%(2*np.pi))
    lam[np.isclose(lam,0,atol=1e-6)]=0
    
    cotl=1/np.tan(lam/2)
    cotal=1/np.tan((aprime-lam)/2)
    
    L = np.where(np.sin(lam)!=0,sinl*np.sin(lam-aprime)*np.log(cotl*cotal),0)
    L += cosl+np.cos(lam-aprime)
    L[np.isclose(L,0)]=0
    L[L!=0] *= (np.pi*a*b*c*Ts*To/T)[L!=0]
    L[L==0] = 1e-15
    return(np.abs(L))

In [304]:
def MCMC(x0,nwalk=10,nminsteps=10**3,nmaxsteps=10**5,nconv=20,step=1,logfunc=None,
         args=None,convergence_func=None,conv_args=None):
    from numpy import matlib
    x0 = np.array(x0)
    dims = np.size(x0)

    x = np.matlib.repmat(np.array(x0),nwalk,1)+np.random.normal(0,0.01,(nwalk,dims))
    fnow = logfunc(x,args)
 
    chain = []
    nsteps = 0
    converged = False
    print_val = True
    sqrt_dist=randDraw(sqrtpdf,2,a=0,b=2.01,dn=0.01)
    while not converged or (nsteps < nminsteps):
        rand = np.random.randint(1,nwalk,size=nwalk)
        j = (np.arange(nwalk)+rand)%nwalk
        xj = x[j]
        z = step*sqrt_dist(nwalk)       

        xtry = xj+np.multiply(np.matlib.repmat(z,dims,1).T,(x-xj))
        ftry = logfunc(xtry,args)
 
        logp = (dims-1)*np.log(z)+ftry-fnow
        prob = np.matlib.repmat(logp,dims,1).T
        rand = np.matlib.repmat(np.log(np.random.uniform(size=nwalk)),dims,1).T       

        x = np.where(rand<=prob,xtry,x)
        chain.append(x)
        fnow = logfunc(x,args)
       
        nsteps += 1
        if nsteps%nconv == 0: converged = convergence_func(chain,conv_args)
        if converged and print_val:
            print("Converged at N = %s"%nsteps)
            print_val = False
        if nsteps > nmaxsteps: break
    return(np.reshape(chain,[-1,dims]))

In [305]:
def lnL(x,y,sigy):
    vary=sigy**2
    L=-0.5*(np.sum((x-y)**2/vary,axis=-1)*np.pi*np.sum(vary))
    return(L)

def min_wrapper(x,p,theta,phi,psi,mag,sigmag,alpha,beta,a,b,c):
    deltaV=x[0]
    betainit=x[1]

    sim=deltaV-2.5*np.log10(evolving_axis_lightcurve(theta,phi,psi,a,b,c,alpha,beta))

    return(-1*lnL(sim,mag,sigmag))

def lnprior(params,bounds):
    '''
    Params is an array of (n,ndims) shape. 
    Bounds is a list of (min,max) tuples
    
    Returns (n,) array of ln(prior) outputs
    
    '''
    bounds=np.array(bounds)
    
    out=np.zeros(params.shape[0])
    
    out+=np.where(np.any(params<bounds[:,0],axis=-1),-1e10,0)
    out+=np.where(np.any(params>bounds[:,1],axis=-1),-1e10,0)
    
    return(out)
        
def MCMCwrapper(x,args):
    from scipy.optimize import minimize
    period=x[:,0]
    theta=x[:,1]
    phi=x[:,2]
    psi=x[:,3]
    
    mag,sigmag,alpha,time,a,b,c,bounds,deltaV,betainit=args
    
    PER,TIME=np.meshgrid(period,time,indexing='ij')
    beta=2*np.pi*(TIME%PER)/PER
    
    simprior=lnprior(x,bounds)
    
    inds=np.where(simprior>=0)[0]
    
    curves=np.zeros((period.size,time.size))
    
    Lml=evolving_axis_lightcurve(theta[inds],phi[inds],psi[inds],a,b,c,alpha,beta[inds,:]-betainit)
    
    curves[inds,:]=deltaV-2.5*np.log10(Lml)
        
    return(lnL(curves,mag,sigmag)+simprior)

In [306]:
x=np.array([[7.3,0,1,1],[7.7,np.pi/2,2,0]])
bounds=[(7,9),(0,1),(-1,1),(0,np.pi)]
args=(np.ones(100),np.ones(100),np.linspace(0,np.pi,100),np.linspace(0,10,100),19,115,111,bounds,33,np.pi/2)
MCMCwrapper(x,args)

array([-9.68444596e+06, -1.00000157e+10])